In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from SCM import ScalarLinearDecisionModel, NewScalarLinearDecisionModel, SelectionBiasDecisionModel
import graph_helper as graph#line_plots, importance_heatmap, diff_importance_heatmap, categorical_heatmap
import optimization_helper as opt #find_pred_improve_lin_cost, find_real_improve_lin_cost
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


In [ ]:
def confusion(X, Y):
    cm = confusion_matrix(X, Y)
    TN = cm[0][0]
    FP = cm[0][1]
    FN = cm[1][0]
    TP = cm[1][1]

    TNR = TN / (TN + FP)
    FPR = FP / (TN + FP)
    FNR = FN / (TP + FN)
    TPR = TP / (TP + FN)
    
    # Print the rates
    print("True Negative Rate (TNR):", TNR)
    print("False Positive Rate (FPR):", FPR)
    print("False Negative Rate (FNR):", FNR)
    print("True Positive Rate (TPR):", TPR)

def fair_model_comp(params, delta = 0.5):
    data_train = NewScalarLinearDecisionModel(params)
    data_train.generate_basic_data()
    data_train_pd = data_train.get_data_df()
    print("Using both features A and C")
    X_AC = np.column_stack((data_train_pd["A"], data_train_pd["C"]))
    model_AC = LogisticRegression()
    model_AC.fit(X_AC, data_train_pd["Y"])
    line_AC = {'w': model_AC.coef_.flatten(), 'b': model_AC.intercept_[0]} 
    print("Training on A and C:")
    print("score: ", model_AC.score(X_AC, data_train_pd["Y"]))
    print("gaming")
    g_data_AC = gaming_metric(params, data_train, line_AC, delta = delta)
    print("minority: ", g_data_AC[0], ", majority: ",  g_data_AC[1])
    print("improvement")
    i_data_AC = improvement_metric(params, data_train, line_AC, delta = delta)
    print("minority: ", i_data_AC[0], ", majority: ",  i_data_AC[1])
    '''print("Minority:")
    print(confusion(data_train_pd["Y"][data_train_pd["S"] == 0], model_AC.predict(X_AC[data_train_pd["S"] == 0])))
    print("Majority:")
    print(confusion(data_train_pd["Y"][data_train_pd["S"] == 1], model_AC.predict(X_AC[data_train_pd["S"] == 1])))
    '''
    print("Using only feature C")
    X_C = np.array([data_train_pd["C"]]).T
    model_C = LogisticRegression()
    model_C.fit(X_C, data_train_pd["Y"])
    c_boundary = data_train.get_real_boundary(s=1, u=0) #s doesn't matter
    line_C = {'w': np.array([0, c_boundary]), 'b': 0} 
    print(c_boundary)
    #line_C = {'w': np.array([0, model_AC.coef_.flatten()[1]]), 'b': model_C.intercept_[0] + model_AC.coef_.flatten()[0]} 
    print("Training just on C:")
    print("score: ", model_C.score(X_C, data_train_pd["Y"]))
    print("improvement")
    i_data_C = improvement_metric(params, data_train, line_C, delta = delta)
    print("minority: ", i_data_C[0], ", majority: ",  i_data_C[1])
    print("gaming")
    g_data_C = gaming_metric(params, data_train, line_C, delta = delta)
    print("minority: ", g_data_C[0], ", majority: ",  g_data_C[1])
    
    '''print("Minority:")
    print(confusion(data_train_pd["Y"][data_train_pd["S"] == 0], model_C.predict(X_C[data_train_pd["S"] == 0])))
    print("Majority:")
    print(confusion(data_train_pd["Y"][data_train_pd["S"] == 1], model_C.predict(X_C[data_train_pd["S"] == 1])))
    #Errors
    print("Minority Error:")
    print(np.mean(np.abs(model_C.predict(X_C[data_train_pd["S"] == 0]) - model_AC.predict(X_AC[data_train_pd["S"] == 0]))))
    print("Majority Error:")
    print(np.mean(np.abs(model_C.predict(X_C[data_train_pd["S"] == 1]) - model_AC.predict(X_AC[data_train_pd["S"] == 1]))))
    #print("[[true negative, false positive],")
    #print("[false negative, true positive]]")'''
    
    
    